# Model training lab

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

## Prerequisits

In [1]:
# Imports
import os
import warnings
import tools
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Bidirectional
from matplotlib import pyplot

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Root CSV files directory
dirname = "./data/"

# Constant frame count.
frames = 100

## Preparation Stage
### Load data and normalize
For training it's required to extend/reduce every dataset to n frames, where n is `frames`.

In [2]:
listfile = os.listdir(dirname)
data = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True) 
        data.append((wordname, content))


In [ ]:
# Print the content of the first dataframe
data[10][0]

### Split data
Split the dataset up into the following segments:
1. Training Data: 60%
2. Validation Data: 20%
3. Test Data: 20%


In [3]:
features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train, x_val, y_train, y_val = train_test_split(features, labels, test_size=0.40, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.50, random_state=42)

In [ ]:
def printCountDataSets(dataset):
    wortCounter = []
    #Liste mit einmaligen Labels erstellen
    labels = sorted(set(dataset), key=dataset.index)
    #Liste nochmal Alphabetisch sortieren
    labels = sorted(labels)
    for label in labels:
        wortCounter.append(0)
    for row in dataset:
        for i in range(len(labels)):
            if str(labels[i]).startswith(row):
                wortCounter[i] += 1
    for i in range(len(labels)):
        print(labels[i], ': ', wortCounter[i], end =";  ")
    print(' ')
        
        
print('Alle Klassensätze')
printCountDataSets(labels)
print('Trainings Klassensätze')
printCountDataSets(y_train)
print('Test Klassensätze')
printCountDataSets(y_test) 
print('Validation Klassensätze')
printCountDataSets(y_val)  

In [ ]:
# Display data distribution
print("Total:", len(labels))
print("Training:", len(y_train), len(y_train) / len(labels) * 100)
print("Val:", len(y_val), len(y_val) / len(labels) * 100)
print("Test:", len(y_test), len(y_test) / len(labels) * 100)

### Tokenize (One Hot)

In [4]:
tokenizer = tools.tokenize(dirname)
print(tokenizer.word_index)
encoded_train=tokenizer.texts_to_sequences([y_train])[0]
encoded_val=tokenizer.texts_to_sequences([y_val])[0]
encoded_test=tokenizer.texts_to_sequences([y_test])[0]
y_train = to_categorical(encoded_train)
y_val = to_categorical(encoded_val)
y_test = to_categorical(encoded_test)
print(y_train)

{'welt': 1, 'computer': 2, 'hallo': 3, 'deutschland': 4}
[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]]


In [5]:
# Making numpy arrays
x_train=np.array(x_train)
y_train=np.array(y_train)
x_val=np.array(x_val)
y_val=np.array(y_val)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [ ]:
print(x_train)

## Training Stage
Configure the model and train it.

Metrics:
<div float="right">
    <img src="assets/accuracy.png" width="400"> 
    <img src="assets/precision_recall_formula.png" width="400">
</div>
<img src="assets/precision_recall.png" width="1000">

### Configure model
Configures the model with the specified parameters.

#### LSTM

In [ ]:
model = Sequential()
model.add(layers.LSTM(256, return_sequences=True,
               input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(layers.LSTM(64, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(layers.LSTM(32))  # return a single vector of dimension 32
#model.add(layers.LSTM(16))  # return a single vector of dimension 32
model.add(layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

### or
#### Bidirectional LSTM

In [ ]:
model = Sequential()
model.add(Bidirectional(layers.LSTM(64, return_sequences=True), input_shape=(100, 86)))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

### or
#### Simple RNN

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=64, input_shape=(200, 42)))
model.add(Dense(64, activation="softmax")) #softmax, linear 어떤걸 기준으로 하지
model.add(Dense(128, activation="linear")) #softmax, linear 어떤걸 기준으로 하지
model.add(Dense(21))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

### or
#### Hyperparametertuned LSTM
##### Here it is necessary to install the Keras-Tuner Module by executing:
##### via Conda:
conda install -c conda-forge keras-tuner
##### for pip:
ip install keras-tuner


In [6]:
from kerastuner.tuners import RandomSearch
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
import time

#LOG_DIR = f"{int(time.time())}"
#LOG_DIR = "/home/miguel/ML/SignLanguageRecognition-master/lab/HyperParameterOptimization/"
LOG_DIR = "Optimization_"f"{int(time.time())}"

print(LOG_DIR)

def build_model(hp):
    model = Sequential()
    
    model.add(layers.LSTM(hp.Int("LSTM_input", min_value =32, max_value=256,step=32), 
                            return_sequences=True,
                            input_shape=(x_train.shape[1], x_train.shape[2])))
    
    for i in range(hp.Int("n_layers" , 1, 4)):    
        model.add(layers.LSTM(hp.Int(f"LSTM_{i}_units", min_value =32, max_value=256,step=32),
                                return_sequences=True))
    
    model.add(layers.LSTM(hp.Int(f"LSTM_End", min_value =32, max_value=256,step=32)))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    model.summary()
    return model

Optimization_1586182635


## Search - RandomSearch

In [ ]:
tuner  = RandomSearch(
    build_model,
    objective = "val_accuracy",
    max_trials = 8,
    executions_per_trial = 1,
    directory = LOG_DIR,
    project_name='SignLagnuageModelOptimization'
    )

tuner.search_space_summary()

'''tuner.search(x=x_train, 
                y= y_train,
            epochs=80,
            batch_size=32,
            validation_data=(x_val,y_val))'''

print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())
#model= build_model()
#history=model.fit(x_train,y_train,epochs=80,validation_data=(x_val,y_val),shuffle=False)

## Search - Hyperband

In [ ]:
tuner  = Hyperband(
    build_model,
    objective = "val_accuracy",
    max_epochs=90,
    directory = LOG_DIR,
    project_name='SignLagnuageModelOptimization'
    )

#tuner.search_space_summary()

tuner.search(x=x_train, 
                y= y_train,
            batch_size=32,
            validation_data=(x_val,y_val))

#print(tuner.get_best_hyperparameters()[0].values)
#print(tuner.results_summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 32)           15232     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 32)           8320      
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 32,037
Trainable params: 32,037
Non-trainable params: 0
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 256)          351232    
________________________________

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 96)           70272     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 224)          287616    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 224)          402304    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 128)          180736    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 192)          246528    
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                28800     
_________________________________________________________________
dense (Dense)                (None, 5)                 1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 96)           70272     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 192)          221952    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 160)          225920    
_________________________________________________________________
lstm_3 (LSTM)                (None, 192)               271104    
_________________________________________________________________
dense (Dense)                (None, 5)                 965       
Total params: 790,213
Trainable params: 790,213
Non-trainable params: 0
_________________________________________________________________
Train on 208 samples, validate on 69 samples
Epoch 1/2
208/208 [==============================] - ETA: 31s - loss: 1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 160)          158080    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 64)           57600     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 128)          98816     
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 335,269
Trainable params: 335,269
Non-trainable params: 0
_________________________________________________________________
Train on 208 samples, validate on 69 samples
Epoch 1/2
208/208 [==============================] - ETA: 31s - loss: 1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 96)           70272     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 160)          164480    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 128)          147968    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 160)          184960    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 160)          205440    
_________________________________________________________________
lstm_5 (LSTM)                (None, 160)               205440    
_________________________________________________________________
dense (Dense)                (None, 5)                 8

...

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 64)           38656     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 96)           61824     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                16512     
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 117,157
Trainable params: 117,157
Non-trainable params: 0
_________________________________________________________________
Train on 208 samples, validate on 69 samples
Epoch 3/4
208/208 [==============================] - ETA: 24s - loss: 1.6095 - accuracy: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+ - ETA: 10s - loss: 1.5979 - accuracy: 0.1719 - precision: 0.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 192)          214272    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 224)          373632    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 192)          320256    
_________________________________________________________________
lstm_3 (LSTM)                (None, 192)               295680    
_________________________________________________________________
dense (Dense)                (None, 5)                 965       
Total params: 1,204,805
Trainable params: 1,204,805
Non-trainable params: 0
_________________________________________________________________
Train on 208 samples, validate on 69 samples
Epoch 3/4
208/208 [==============================] - ETA: 32s - los

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 96)           70272     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 192)          221952    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 64)           65792     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
Total params: 391,365
Trainable params: 391,365
Non-trainable params: 0
_________________________________________________________________
Train on 208 samples, validate on 69 samples
Epoch 5/10
208/208 [==============================] - ETA: 30s - loss: 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 128)          110080    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 160)          184960    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                57600     
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
Total params: 352,965
Trainable params: 352,965
Non-trainable params: 0
_________________________________________________________________
Train on 208 samples, validate on 69 samples
Epoch 5/10
208/208 [==============================] - ETA: 24s - loss: 1.6092 - accuracy: 0.3438 - precision: 0.0000e+00 - recall: 0.0000e+ - ETA: 10s - loss: 1.5996 - accuracy: 0.3281 - precision: 0.000

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 64)           38656     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 192)          197376    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 96)           110976    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 224)          287616    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                73984     
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
Total params: 708,933
Trainable params: 708,933
Non-trainable params: 0
__________________________________________________

### Diagnostic Plots

The training history of your LSTM models can be used to diagnose the behavior of your model.

You can plot the performance of your model using the Matplotlib library. For example, you can plot training loss vs test loss as follows:

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

#### Underfit Example
Running this example produces a plot of train and validation loss showing the characteristic of an underfit model. In this case, performance may be improved by increasing the number of training epochs.


<img src="assets/Diagnostic-Line-Plot-Showing-an-Underfit-Model.png" width="400">


Running this example shows the characteristic of an underfit model that appears under-provisioned.
In this case, performance may be improved by increasing the capacity of the model, such as the number of memory cells in a hidden layer or number of hidden layers.

<img src="assets/Diagnostic-Line-Plot-Showing-an-Underfit-Model-via-Status.png" width="400">

#### Good Fit Example
Running the example creates a line plot showing the train and validation loss meeting.
Ideally, we would like to see model performance like this if possible, although this may not be possible on challenging problems with a lot of data.

<img src="assets/Diagnostic-Line-Plot-Showing-a-Good-Fit-for-a-Model.png" width="400">

#### Overfit Example
Running this example creates a plot showing the characteristic inflection point in validation loss of an overfit model.
This may be a sign of too many training epochs.
In this case, the model training could be stopped at the inflection point. Alternately, the number of training examples could be increased.

<img src="assets/Diagnostic-Line-Plot-Showing-an-Overfit-Model.png" width="400">

### Evaluate

In [ ]:
model.evaluate(x=x_test, y=y_test)

### Save model

In [ ]:

model.save("sign_lang_recognition_tuned.h5")